In [22]:
#imports
from datasets import load_from_disk
import pandas as pd

### General

In [248]:
# A lookup dict containing translations from EN to NL
#logic: of sentences can be found: https://github.com/AI4Finance-Foundation/FinGPT/blob/3b0e779e5c1a547ef1900e7bc5ea8ac37cd65d34/fingpt/FinGPT_Benchmark/data/prepare_data.ipynb#L1922 section finred
#FinRED/relations.txt can be found: https://github.com/AI4Finance-Foundation/FinGPT/blob/master/fingpt/FinGPT_Benchmark/benchmarks/finred.py
options_lookup_dict = {
    'product_or_material_produced': 'product/materiaal geproduceerd',
    'product or material produced': 'product/materiaal geproduceerd',
    'product/material produced': 'product/materiaal geproduceerd',
    'manufacturer': 'fabrikant',
    'distributed_by':   'verdeeld door',
    'distributed by':   'verdeeld door',
    'industry': 'industrie',
    'position_held':    'positie bekleed',
    'position held':    'positie bekleed',
    'original_broadcaster': 'originele omroep',
    'original broadcaster': 'originele omroep',
    'owned_by': 'bezeten door',
    'owned by': 'bezeten door',
    'founded_by':   'opgericht door',
    'founded by':   'opgericht door',
    'distribution_format':  'distributieformaat',
    'distribution format':  'distributieformaat',
    'headquarters_location':    'hoofdkantoorlocatie',
    'headquarters location':    'hoofdkantoorlocatie',
    'stock_exchange':   'effectenbeurs',
    'stock exchange':   'effectenbeurs',
    'currency': 'valuta',
    'parent_organization':  'moederorganisatie',
    'parent organization':  'moederorganisatie',
    'chief_executive_officer':  'chief executive officer',
    'chief executive officer':  'chief executive officer',
    'director_/_manager':   'directeur/manager',
    'director/manager':   'directeur/manager',
    'owner_of': 'eigenaar van',
    'owner of': 'eigenaar van',
    'operator': 'operator',
    'member_of':    'lid van',
    'member of':    'lid van',
    'employer': 'werkgever',
    'chairperson':  'voorzitter',
    'platform': 'platform',
    'subsidiary':   'dochteronderneming',
    'legal_form':   'rechtsvorm',
    'legal form':   'rechtsvorm',
    'publisher':    'uitgever',
    'developer':    'ontwikkelaar',
    'brand':    'merk',
    'business_division':    'bedrijfsdivisie',
    'business division':    'bedrijfsdivisie',
    'location_of_formation':    'locatie van ontstaan',
    'location of formation':    'locatie van ontstaan',
    'creator':  'maker',
}

### FinRed

In [157]:
pattern_1 = r"Utilize the input text as a context reference, choose the right relationship between .* and .* Options: .*"
pattern_2 = r"What is the relationship between .* and .* in the context of the input sentence. Choose an answer from: .*"
pattern_3 = r"Given phrases that describe the relationship between two words/phrases as options, extract the word/phrase pair and the corresponding lexical relationship between them from the input text. The output format should be \"relation1: word1, word2; relation2: word3, word4\". Options: .*"
pattern_4 = r"Given the input sentence, please extract the subject and object containing a certain relation in the sentence according to the following relation types, in the format of \"relation1: word1, word2; relation2: word3, word4\". Relations include: .*"


finred_lookup = {
    #patter_3
    "Given phrases that describe the relationship between two words/phrases as options, extract the word/phrase pair and the corresponding lexical relationship between them from the input text. The output format should be \"relation1: word1, word2; relation2: word3, word4\"" : "Gegeven zinnen die de relatie tussen twee woorden/zinnen beschrijven als opties, haal het woord/zinspaar en de overeenkomstige lexicale relatie tussen hen uit de invoertekst. het formaat van het resultaat moet zijn \"relatie1: woord1, woord2; relatie2: woord3, woord4\"",
    #pattern_4
    "Given the input sentence, please extract the subject and object containing a certain relation in the sentence according to the following relation types, in the format of \"relation1: word1, word2; relation2: word3, word4\". Relations include": "Gegeven de invoerzin, gelieve het onderwerp en het object te extraheren dat een bepaalde relatie in de zin bevat volgens de volgende soorten relaties, in het formaat van \"relatie1: woord1, woord2; relatie2: woord3, woord4\". Relaties zijn onder andere",
    #patter_1
    "Utilize the input text as a context reference, choose the right relationship between":"Gebruik de invoertekst als een contextreferentie, kies de juiste relatie tussen",
    "and": "en",
    "Options": "Opties",
    "from the options": "uit de opties",
    #patter_2
    "What is the relationship between": "Wat is de relatie tussen",
    "in the context of the input sentence. Choose an answer from": "in de context van de invoerzin. Kies een antwoord uit",
}

In [121]:
# Analysis
df.shape[0] == df['instruction'].str.match(pattern_1).sum() + df['instruction'].str.match(pattern_2).sum() + df['instruction'].str.match(pattern_3).sum() + df['instruction'].str.match(pattern_4).sum()


True

In [179]:
datasetdict = load_from_disk("../data/FinGPT/fingpt-finred")
for split in datasetdict:
    df = datasetdict[split].to_pandas()

    for english, dutch in finred_lookup.items():
        df['instruction'] = df['instruction'].str.replace(english, dutch)

    for english, dutch in options_lookup_dict.items():
        df['instruction'] = df['instruction'].str.replace(english, dutch)

    df['instruction'] = df['instruction'].str.replace("enere", "andere")

    datasetdict[split] = Dataset.from_pandas(df)

In [181]:
datasetdict.save_to_disk("../data/finget/fingpt-finred-nl")


Saving the dataset (0/1 shards):   0%|          | 0/27558 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5112 [00:00<?, ? examples/s]

### FinRed cls

In [209]:
datasetdict = load_from_disk("../data/FinGPT/fingpt-finred-cls")
df = datasetdict['train'].to_pandas()
df['instruction'].str.match(pattern_1).sum() + df['instruction'].str.match(pattern_2).sum() + df['instruction'].str.match(pattern_3).sum() + df['instruction'].str.match(pattern_4).sum()


0

In [205]:
template_1 = r"Utilize the input text as a context reference, choose the right relationship between .* and .* from the options.\nOptions"
template_2 = r"Refer to the input text as context and select the correct relationship between .* and .* from the available options.\nOptions:"
template_3 = r"Take context from the input text and decide on the accurate relationship between .* and .* from the options provided.\nOptions:"
template_4 = r"What is the relationship between .* and .* in the context of the input sentence.\nOptions:"
template_5 = r"In the context of the input sentence, determine the relationship between .* and .*.\nOptions"
template_6 = r"Analyze the relationship between .* and .* within the context of the input sentence.\nOptions"
df['instruction'].str.match(template_6).sum()


8079

#### Analysis

In [208]:
sum = 0
for template in [template_1, template_2, template_3, template_4, template_5, template_6]:
    sum += df['instruction'].str.match(template).sum()

sum == df.shape[0]

True

In [227]:
finred_cls_lookup = {
"Utilize the input text as a context reference, choose the right relationship between" : "Gebruik de invoertekst als contextreferentie, kies de juiste relatie tussen",
"from the options.\nOptions" : "uit de opties.\nOpties",
"Refer to the input text as context and select the correct relationship between": "Verwijs naar de invoertekst als context en selecteer de juiste relatie tussen",
"from the available options.\nOptions:" : "uit de beschikbare opties.\nOpties",
"Take context from the input text and decide on the accurate relationship between" : "Haal de context uit de invoertekst en beslis over de nauwkeurige relatie tussen",
"from the options provided.\nOptions:" : "uit de verstrekte opties\nOpties",
"What is the relationship between" : "Wat is de relatie tussen",
"in the context of the input sentence.\nOptions:" : "in de context van de invoerzin.\nOpties",
"In the context of the input sentence, determine the relationship between" : "In de context van de invoerzin, bepaal de relatie tussen",
"Analyze the relationship between" : "Analyseer de relatie tussen",
"within the context of the input sentence.\nOptions" : "binnen de context van de invoerzin.\nOpties",
"and": "en",
"\Options": "\Opties",
}


In [228]:
datasetdict = load_from_disk("../data/FinGPT/fingpt-finred-cls")
for split in datasetdict:
    df = datasetdict[split].to_pandas()

    for english, dutch in finred_cls_lookup.items():
        df['instruction'] = df['instruction'].str.replace(english, dutch)

    for english, dutch in options_lookup_dict.items():
        df['instruction'] = df['instruction'].str.replace(english, dutch)

    datasetdict[split] = Dataset.from_pandas(df)

In [230]:
datasetdict.save_to_disk("../data/finget/fingpt-finred-cls-nl")

Saving the dataset (0/1 shards):   0%|          | 0/48474 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8928 [00:00<?, ? examples/s]

### FinRed re

In [231]:
datasetdict = load_from_disk("../data/FinGPT/fingpt-finred-re")
df = datasetdict['train'].to_pandas()

In [240]:
template_1 = r"Given phrases that describe the relationship between two words/phrases as options, extract the word/phrase pair and the corresponding lexical relationship between them from the input text. The output format should be \"relation1: word1, word2; relation2: word3, word4\"."
template_2 = r"Given the input sentence, please extract the subject and object containing a certain relation in the sentence according to the following relation types, in the format of \"relation1: word1, word2; relation2: word3, word4\". Relations include:"

df["instruction"].str.match(template_1).sum() + df["instruction"].str.match(template_2).sum() == df.shape[0]

True

In [243]:
finred_re_lookup = {
    "Given phrases that describe the relationship between two words/phrases as options, extract the word/phrase pair and the corresponding lexical relationship between them from the input text. The output format should be \"relation1: word1, word2; relation2: word3, word4\"." : "Gegeven zinnen die de relatie tussen twee woorden/zinnen beschrijven als opties, extraheren vanuit de invoertekst het woord/zinspaar en de overeenkomstige lexicaire relatie tussen hen. Het uitvoerformaat moet zijn \"relatie1: woord1, woord2; relatie2: woord3, woord4\".",
    "Given the input sentence, please extract the subject and object containing a certain relation in the sentence according to the following relation types, in the format of \"relation1: word1, word2; relation2: word3, word4\". Relations include:" : "Gegeven de invoerzin, gelieve het onderwerp en object te extraheren dat een bepaalde relatie in de zin bevat volgens de volgende soorten relaties, in het formaat van \"relatie1: woord1, woord2; relatie2: woord3, woord4\". Relaties omvatten:"
}

In [244]:
datasetdict = load_from_disk("../data/FinGPT/fingpt-finred-re")
for split in datasetdict:
    df = datasetdict[split].to_pandas()

    for english, dutch in finred_re_lookup.items():
        df['instruction'] = df['instruction'].str.replace(english, dutch)

    for english, dutch in options_lookup_dict.items():
        df['instruction'] = df['instruction'].str.replace(english, dutch)

    datasetdict[split] = Dataset.from_pandas(df)

In [247]:
datasetdict.save_to_disk("../data/finget/fingpt-finred-re-nl")

Saving the dataset (0/1 shards):   0%|          | 0/11400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2136 [00:00<?, ? examples/s]